In [1]:
from sklearn.naive_bayes import MultinomialNB
import os
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import FastTextFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import USEFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import GloVeFeatureGeneration
from sklearn.metrics import f1_score
import csv
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import collections
from imblearn.over_sampling import SMOTE

In [2]:
class NaiveBayes:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.nb = MultinomialNB()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.nb.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.nb.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

In [4]:
column_headings = ["Morbidity Class", "NB_Macro F1", "NB_Micro F1"]

with open("./results/word-embeddings-features/performance_NB_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [5]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).matrix_gen()
    X = np.abs(np.average(X, axis=1))
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # print(morbidity)
        # smote = SMOTE(random_state=42,k_neighbors=2)
        # X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        # X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(66, 494, 300) (66,) Counter({0.0: 63, 1.0: 3})
Macro F1 score: 0.837062937062937 and Micro F1 Score 0.9523809523809523
(62, 495, 300) (62,) Counter({0.0: 35, 1.0: 27})
Macro F1 score: 0.3577272727272727 and Micro F1 Score 0.5642857142857143
(12, 494, 300) (12,) Counter({1.0: 12})
Macro F1 score: 1 and Micro F1 Score 1
(66, 495, 300) (66,) Counter({0.0: 58, 1.0: 8})
Macro F1 score: 0.6665501165501164 and Micro F1 Score 0.880952380952381
(63, 495, 300) (63,) Counter({1.0: 35, 0.0: 28})
Macro F1 score: 0.35181818181818186 and Micro F1 Score 0.5547619047619048
(66, 495, 300) (66,) Counter({0.0: 56, 1.0: 10})
Macro F1 score: 0.655128205128205 and Micro F1 Score 0.85
(56, 495, 300) (56,) Counter({0.0: 49, 1.0: 7})
Macro F1 score: 0.6662626262626263 and Micro F1 Score 0.8800000000000001
(67, 495, 300) (67,) Counter({0.0: 62, 1.0: 5})
Macro F1 score: 0.7777622377622377 and Micro F1 Score 0.9214285714285715
(59, 495, 300) (59,) Counter({0.0: 43, 1.0: 16})
Macro F1 score: 0.4647727272727272 and

In [6]:
with open("./results/word-embeddings-features/performance_NB_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = GloVeFeatureGeneration(train_preprocessed_df, morbidity).matrix_gen()
    X = np.abs(np.average(X, axis=1))
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Asthma
Macro F1 score: 0.9404616298733945 and Micro F1 Score 0.9442307692307692
CAD
CAD
Macro F1 score: 0.7226370851370851 and Micro F1 Score 0.7428571428571428
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Depression
Macro F1 score: 0.6772832396361809 and Micro F1 Score 0.6984848484848485
Diabetes
Diabetes
Macro F1 score: 0.39190476190476187 and Micro F1 Score 0.4428571428571428
Gallstones
Gallstones
Macro F1 score: 0.6565734265734265 and Micro F1 Score 0.6696969696969697
GERD
GERD
Macro F1 score: 0.7157029082029082 and Micro F1 Score 0.7233333333333334
Gout
Gout
Macro F1 score: 0.885324871207224 and Micro F1 Score 0.896153846153846
Hypercholesterolemia
Hypercholesterolemia
Macro F1 score: 0.6049242424242424 and Micro F1 Score 0.6180555555555556
Hypertension
Hypertension
Macro F1 score: 0.7034398934398934 and Micro F1 Score 0.7388888888888888
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
OA
Macro F1 score: 0.8064359251859251 and Micro F1 Score 0.8121212

In [7]:
with open("./results/word-embeddings-features/performance_NB_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FastTextFeatureGeneration(train_preprocessed_df, morbidity).matrix_gen()
    X = np.abs(np.average(X, axis=1))
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # print(morbidity)
        # smote = SMOTE(random_state=42,k_neighbors=2)
        # X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        # X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Macro F1 score: 0.837062937062937 and Micro F1 Score 0.9523809523809523
CAD
Macro F1 score: 0.3577272727272727 and Micro F1 Score 0.5642857142857143
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Macro F1 score: 0.6665501165501164 and Micro F1 Score 0.880952380952381
Diabetes
Macro F1 score: 0.35181818181818186 and Micro F1 Score 0.5547619047619048
Gallstones
Macro F1 score: 0.655128205128205 and Micro F1 Score 0.85
GERD
Macro F1 score: 0.6662626262626263 and Micro F1 Score 0.8800000000000001
Gout
Macro F1 score: 0.7777622377622377 and Micro F1 Score 0.9214285714285715
Hypercholesterolemia
Macro F1 score: 0.4647727272727272 and Micro F1 Score 0.7266666666666667
Hypertension
Macro F1 score: 0.5195202020202021 and Micro F1 Score 0.74
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
Macro F1 score: 0.6110372960372961 and Micro F1 Score 0.8619047619047618
Obesity
Macro F1 score: 0.34822510822510816 and Micro F1 Score 0.5619047619047619
OSA
Macro F1 score: 0.6037

In [8]:
with open("./results/word-embeddings-features/performance_NB_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = USEFeatureGeneration(train_preprocessed_df, morbidity).matrix_gen()

    X = np.abs(np.average(X, axis=1))
    #X = X.reshape(-1, 1)
    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # print(morbidity)
        # smote = SMOTE(random_state=42,k_neighbors=min(1, len(X)-1))
        # X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        # X, Y =  X_train_resampled, y_train_resampled
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma


2023-05-04 19:20:25.655055: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Macro F1 score: 0.837062937062937 and Micro F1 Score 0.9523809523809523
CAD
Macro F1 score: 0.7580555555555556 and Micro F1 Score 0.7714285714285715
CHF
Macro F1 score: 1 and Micro F1 Score 1
Depression
Macro F1 score: 0.4785878010878011 and Micro F1 Score 0.7261904761904762
Diabetes
Macro F1 score: 0.5945238095238096 and Micro F1 Score 0.6166666666666666
Gallstones
Macro F1 score: 0.5716083916083916 and Micro F1 Score 0.7547619047619047
GERD
Macro F1 score: 0.5551515151515152 and Micro F1 Score 0.8400000000000001
Gout
Macro F1 score: 0.7732750582750583 and Micro F1 Score 0.9071428571428571
Hypercholesterolemia
Macro F1 score: 0.5164069264069264 and Micro F1 Score 0.6466666666666667
Hypertension
Macro F1 score: 0.6259920634920635 and Micro F1 Score 0.6866666666666666
Hypertriglyceridemia
Macro F1 score: 1 and Micro F1 Score 1
OA
Macro F1 score: 0.6292773892773893 and Micro F1 Score 0.8214285714285715
Obesity
Macro F1 score: 0.6426587301587301 and Micro F1 Score 0.6714285714285715
OSA
M